In [ ]:
#Fisher Score- Chisquare Test For Feature Selection

In [22]:
import pandas as pd
import seaborn as sns
df = sns.load_dataset('titanic')

In [23]:
# This score should be used to avaluate categorial variables in a classification task

In [24]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [26]:
#Categorial variables
df = df[['sex','embarked','alone','pclass','survived']]
df.head()

,sex,embarked,alone,pclass,survived
0,male,S,False,3,0
1,female,C,False,1,1
2,female,S,True,3,1
3,female,S,False,1,1
4,male,S,True,3,0


In [27]:
### Let's perform Label encoding on sex column
import numpy as np
df['sex']=np.where(df['sex']=='male',1,0)
### Let's perform Label encoding on embarked
original_label = {k: i for i, k in enumerate(df['embarked'].unique(),0)}
df['embarked'] = df['embarked'].map(original_label) # replace key on value

In [28]:
df.head()

,sex,embarked,alone,pclass,survived
0,1,0,False,3,0
1,0,1,False,1,1
2,0,0,True,3,1
3,0,0,False,1,1
4,1,0,True,3,0


In [29]:
original_label

{'S': 0, 'C': 1, 'Q': 2, nan: 3}

In [30]:
### Let's perform label encoding on alone
df['alone'] = np.where(df['alone']==True,1,0)

In [31]:
df.head()

,sex,embarked,alone,pclass,survived
0,1,0,0,3,0
1,0,1,0,1,1
2,0,0,1,3,1
3,0,0,0,1,1
4,1,0,1,3,0


In [32]:
### train Test split is usually done to avoid overfitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['sex','embarked','alone','pclass']],
                                                   df['survived'],
                                                    test_size=0.3,
                                                    random_state=0)

In [33]:
X_train.head()

,sex,embarked,alone,pclass
857,1,0,1,1
52,0,1,0,1
386,1,0,0,3
124,1,0,0,1
578,0,1,0,3


In [34]:
X_train.isnull().sum()

sex         0
embarked    0
alone       0
pclass      0
dtype: int64

In [35]:
## Perform chi2 test
# chi 2 return 2 values
# Fscore and the p-value
from sklearn.feature_selection import chi2
f_p_values=chi2(X_train, y_train)

In [36]:
f_p_values

(array([63.55447864, 11.83961845,  9.03328564, 21.61080949]),
 array([1.55992554e-15, 5.79837058e-04, 2.65107556e-03, 3.33964360e-06]))

In [ ]:
#The greater the F, the more important the function. The smaller the p value, the more important the function

In [37]:
p_values = pd.Series(f_p_values[1])
p_values.index=X_train.columns
p_values

sex         1.559926e-15
embarked    5.798371e-04
alone       2.651076e-03
pclass      3.339644e-06
dtype: float64

In [38]:
p_values.sort_index(ascending=False)

sex         1.559926e-15
pclass      3.339644e-06
embarked    5.798371e-04
alone       2.651076e-03
dtype: float64

In [ ]:
# Observation
# Sex column is the most important column when compared
# to the output feature Survived 